In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# In the real world
# 1. run singleVesselUQSA.xml with standard simulator?
# 2. run VascularPolynomialChaos.py  on the singleVesselUQSA.xml to produce vpcCase
# 3. run VascularPolynomialChaos.py on the resulting vpc.xml to evaluate all

import sys, os
import shutil
import numpy as np
import math
import starfish.UtilityLib.moduleFilePathHandler as mFPH
import starfish.UtilityLib.moduleXML as mXML
import starfish.VascularPolynomialChaosLib.classDistributionManager as cDistMng
import starfish.VascularPolynomialChaosLib.moduleFilePathHandlerVPC as mFPH_VPC
import starfish.VascularPolynomialChaosLib.moduleBatchSimulationManager as mBSM
import starfish.VascularPolynomialChaosLib.classUqsaCase as cUqsaCase
import starfish.VascularPolynomialChaosLib.classRandomInput as cRI

In [ ]:
networkName = "singleVesselUQSA"
dataNumber = "exm"

# create template configuration
configurationFilePathTemplate = mFPH_VPC.getFilePath('uqsaCaseTemplateFile', networkName, dataNumber, 'read')
uqsaCase = cUqsaCase.UqsaCase()
uqsaCase.loadXMLFile(configurationFilePathTemplate)

# TODO: check if file exits and ask if it should be overwritten        
uqsaCaseFile = mFPH_VPC.getFilePath('uqsaCaseXmlFile', networkName, dataNumber, 'write')
uqsaCase.writeXMLFile(uqsaCaseFile)
# copy network file
toCopyFile = mFPH.getFilePath('networkXmlFile', networkName, 'xxx','write')
destinationFile = mFPH_VPC.getFilePath('vpcNetworkXmlFile', networkName, dataNumber,'write')
shutil.copy(toCopyFile, destinationFile)

In [ ]:

networkName = "singleBifurcation_template"
dataNumber = "exm"

In [ ]:
uqsaCase = cUqsaCase.UqsaCase() 
uqsaCase.loadXMLFile(uqsaCaseFile)
uqsaCase.initialize(networkName,dataNumber)

In [ ]:
loi = uqsaCase.locationOfInterestManager.locationsOfInterest['exampleLocation']

In [ ]:
loi.queryLocation

In [ ]:
qoi = loi.quantitiesOfInterest['Flow']

In [ ]:
vpcNetworkXmlFile = mFPH_VPC.getFilePath('vpcNetworkXmlFile', networkName, dataNumber, 'write')
vascularNetwork = mXML.loadNetworkFromXML(networkName, dataNumber, networkXmlFile = vpcNetworkXmlFile)
# TODO: steps 1.1 and 1.2 should be entirely external to the UQSA core
    

In [ ]:
vascularNetwork = mXML.loadNetworkFromXML(networkName, 'xxx')

In [ ]:
vascularNetwork.boundaryConditions[0][0]

In [ ]:
import starfish.SolverLib.class1DflowSolver as c1dFS
flowSolver = c1dFS.FlowSolver(vascularNetwork, quiet=True)
flowSolver.solve()

In [ ]:
vessel =vascularNetwork.vessels[1]
plt.figure()
plt.plot(vascularNetwork.simulationTime[1:], vessel.Qsol[1:,30]*1e6)
plt.plot(vascularNetwork.simulationTime[1:], vessel.Qsol[1:,20]*1e6)
plt.ylabel("Flow")

plt.figure()
plt.plot(vascularNetwork.simulationTime[1:], vessel.Psol[1:,30]/133.32)
plt.plot(vascularNetwork.simulationTime[1:], vessel.Psol[1:,20]/133.32)
plt.ylabel("Pressure")
plt.show()

In [ ]:
rim = vascularNetwork.randomInputManager


In [ ]:
vessel = vascularNetwork.vessels[1]
vessel.A0

In [ ]:
vessel.initialize(vascularNetwork.globalFluid)

In [ ]:
vessel.As

In [ ]:
# Create a random input
Z = cRI.ParametricRandomInput()
Z.a = vessel.As
pct_unc = 0.05
Z.b = pct_unc*vessel.As
Z.distributionType = 'Uniform'
Z.parameter = "vessel_1_As"

In [ ]:
rim.randomInputs['Z3']=Z

In [ ]:
rim.printOutInfo()

In [ ]:
vascularNetwork.initialize()
vascularNetwork.vessels

In [ ]:
# Save changes
uqsaCase.writeXMLFile(uqsaCaseFile)
mXML.writeNetworkToXML(vascularNetwork, dataNumber=dataNumber, networkXmlFile=vpcNetworkXmlFile)
#vascularNetwork.writeXMLFile(vpcNetworkXmlFile)

In [ ]:
uqsaCase = cUqsaCase.UqsaCase() 
uqsaCase.loadXMLFile(uqsaCaseFile)
uqsaCase.initialize(networkName,dataNumber)


vpcNetworkXmlFileSol = os.path.join(*[".", "singleVesselUQSA", "vascularPolynomialChaos_tst",  "singleVesselUQSA_vpc_tst.xml"])
vpcNetworkXmlFile = mFPH_VPC.getFilePath('vpcNetworkXmlFile', networkName, dataNumber, 'write')
if not os.path.exists(os.path.split(vpcNetworkXmlFileSol)[0]):
    os.makedirs(os.path.split(vpcNetworkXmlFileSol)[0])
shutil.copy(vpcNetworkXmlFileSol, vpcNetworkXmlFile)

# 1.2 load vascular network file polynomial chaos
vpcNetworkXmlFileSol = os.path.join(*[".", "singleVesselUQSA", "vascularPolynomialChaos_tst",  "singleVesselUQSA_vpc_tst.xml"])
vpcNetworkXmlFile = mFPH_VPC.getFilePath('vpcNetworkXmlFile', networkName, dataNumber, 'write')
if not os.path.exists(os.path.split(vpcNetworkXmlFileSol)[0]):
    os.makedirs(os.path.split(vpcNetworkXmlFileSol)[0])
shutil.copy(vpcNetworkXmlFileSol, vpcNetworkXmlFile)

vascularNetwork = mXML.loadNetworkFromXML(networkName, dataNumber, networkXmlFile = vpcNetworkXmlFile)
# TODO: steps 1.1 and 1.2 should be entirely external to the UQSA core

# 1.3 initialized defined random inputs
vascularNetwork.randomInputManager.initialize(vascularNetwork)
